# Wokflow con Full Bayesiana 

##  Big Picture

Se muestra el workflow con la Bayesian Optimization diseñada para que LightGBM maximice la ganancia en pesos argentinos
<br> En la Primera Competencia se maximizo la metrica global ROC AUC,  ahora pasamos a la metrica real que llamamos ganancia y que solo se concentra en los  11000 registros con mayor probabilidad de  {"BAJA+1","BAJA+2"}


Intencionalmente este notebook es un *semi-esqueleto*  al que usted deberá completar reutilizando código que ya generó para la Primera Competencia y nuevo código que resuelva las problemáticas presentes en esta competencia.

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-01 13:37:57 EDT"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,649543,34.7,1439644,76.9,1245341,66.6
Vcells,1206239,9.3,8388608,64.0,1924957,14.7


In [2]:
PARAM <- list()
PARAM$experimento <- "seg-001"
PARAM$semilla_primigenia <- 102191

In [3]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [4]:
Sys.time()
require( "data.table" )

# leo el dataset
dataset <- fread("~/buckets/b1/datasets/competencia_02_crudo.csv.gz" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
  "pos" = .I,
  numero_de_cliente,
  periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 )
]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
  shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente
]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
  ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
  clase_ternaria := "BAJA+1"
]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
  & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
  clase_ternaria := "BAJA+2"
]

# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

rm(dsimple)
gc()
Sys.time()

[1] "2025-11-01 13:37:58 EDT"

Loading required package: data.table



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,758257,40.5,1439644,76.9,1439644,76.9
Vcells,722128601,5509.5,1017371871,7762.0,845986738,6454.4


[1] "2025-11-01 13:38:17 EDT"

In [5]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### Eliminacion de Features

Completar a gusto LUEGO de realizar un analisis exploratorio de datos.
<br> No necesariamente en esta Segunda Competencia conviele eliminar los mismos campos que en la Primera ...

### Data Quality

Se deben reparar los atributos del dataset que para un cierto mes TODOS sus valores son cero.
<br> Relevar en forma muy minuciosa en el dataset cuales son los  <atributo,mes> que estan dañados.
<br> Algunas alternativas de solución son:
* No hacer absolutamente nada, dejar el valor 0 tal cual está, a sabiendas que es incorrecto
* Reemplazar esos valores dañados por  NA
* Interpolar cada valor dañado por el valor del mes previo y el posterior
* Calcularlo a partir de un modelo, libreria  MICE

a este codigo de Data Quality  lo debera escribir usted

### Data Drifting

Se debe corregir el drifting natural que ocurre en loa datos, en particular los datos monetarios que se vieron fuertemente afectados por una alta inflación
<br> Posibles métodos son:
* No hacer absolutamente nada
* Ajuste de valores monetarios por indices del tipo :
   * IPC  Indice de Precios al Consumidor
   * Dolar Oficial
   * Dolar Blue
   * UVA  Unidad de Valor Adquisitivo

a este codigo de Data Drifting lo debera escribir usted

### Feature Engineering Intra-Mes

Crear variables nuevas a partir de las existentes dentro del mismo registro, **sin** ir a buscar información histórica.
<br> El siguiente código es un mínimo ejemplo, agregar nuevos features a gusto

In [6]:
# el mes 1,2, ..12 , podria servir para detectar estacionalidad
dataset[, kmes := foto_mes %% 100]

# creo un ctr_quarter que tenga en cuenta cuando
# los clientes hace 3 menos meses que estan
# ya que seria injusto considerar las transacciones medidas en menor tiempo
dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]
dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5.0]
dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2.0]
dataset[cliente_antiguedad == 3, ctrx_quarter_normalizado := ctrx_quarter * 1.2]

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

[1] "2025-11-01 13:38:18 EDT"

### Feature Engineering Historico

In [7]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [8]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")

In [9]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}

In [10]:
# Feature Engineering Historico
# Creacion de LAGs
setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# lags de orden 2
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by= numero_de_cliente,
  .SDcols= cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

Sys.time()

[1] "2025-11-01 13:40:14 EDT"

In [11]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- FALSE
PARAM$FE_hist$Tendencias$maximo <- FALSE
PARAM$FE_hist$Tendencias$promedio <- FALSE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE

In [12]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 933

[1] "2025-11-01 13:41:19 EDT"

### Feature Engineering a partir de hojas de Random Forest

In [13]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: lightgbm



In [14]:
AgregaVarRandomForest <- function() {

  cat( "inicio AgregaVarRandomForest()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% c( "BAJA+2", "BAJA+1"),
      clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c( "clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
     data = dtrain,
     param = PARAM$FE_rf$lgb_param,
     verbose = -100
  )

  cat( "Fin construccion RandomForest\n" )
  # grabo el modelo, achivo .model
  lgb.save(modelo, file="modelo.model" )

  qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

  periodos <- dataset[ , unique( foto_mes ) ]

  for( periodo in  periodos )
  {
    cat( "periodo = ", periodo, "\n" )
    datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

    cat( "Inicio prediccion\n" )
    prediccion <- predict(
        modelo,
        datamatrix,
        type = "leaf"
    )
    cat( "Fin prediccion\n" )

    for( arbolito in 1:qarbolitos )
    {
       cat( arbolito, " " )
       hojas_arbol <- unique(prediccion[ , arbolito])

       for (pos in 1:length(hojas_arbol)) {
         # el numero de nodo de la hoja, estan salteados
         nodo_id <- hojas_arbol[pos]
         dataset[ foto_mes== periodo, paste0(
            "rf_", sprintf("%03d", arbolito),
             "_", sprintf("%03d", nodo_id)
          ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

       }

       rm( hojas_arbol )
    }
    cat( "\n" )

    rm( prediccion )
    rm( datamatrix )
    gc(verbose= FALSE)
  }

  gc(verbose= FALSE)

  # borro clase01 , no debe ensuciar el dataset
  dataset[ , clase01 := NULL ]

}

In [15]:
# Parametros de Feature Engineering  a partir de hojas de Random Forest

# Estos CUATRO parametros son los que se deben modificar
PARAM$FE_rf$arbolitos= 20
PARAM$FE_rf$hojas_por_arbol= 16
PARAM$FE_rf$datos_por_hoja= 100
PARAM$FE_rf$mtry_ratio= 0.2

# Estos son quasi fijos
PARAM$FE_rf$train$training <- c( 202101, 202102, 202103)

# Estos TAMBIEN son quasi fijos
PARAM$FE_rf$lgb_param <-list(
    # parametros que se pueden cambiar
    num_iterations = PARAM$FE_rf$arbolitos,
    num_leaves  = PARAM$FE_rf$hojas_por_arbol,
    min_data_in_leaf = PARAM$FE_rf$datos_por_hoja,
    feature_fraction_bynode  = PARAM$FE_rf$mtry_ratio,

    # para que LightGBM emule Random Forest
    boosting = "rf",
    bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
    bagging_freq = 1.0,
    feature_fraction = 1.0,

    # genericos de LightGBM
    max_bin = 31L,
    objective = "binary",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    force_row_wise = TRUE,
    verbosity = -100,
    max_depth = -1L,
    min_gain_to_split = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1 = 0.0,
    lambda_l2 = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance = FALSE,
    scale_pos_weight = 1.0,

    drop_rate = 0.1,
    max_drop = 50,
    skip_drop = 0.5,

    extra_trees = FALSE
  )

In [16]:
# Feature Engineering agregando variables de Random Forest
#  aqui es donde se hace el trabajo
AgregaVarRandomForest()

Sys.time()

inicio AgregaVarRandomForest()
Fin construccion RandomForest
periodo =  201901 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201902 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201903 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201904 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201905 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201906 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201907 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201908 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7

[1] "2025-11-01 13:45:49 EDT"

In [17]:
ncol(dataset)
colnames(dataset)

[1] 1254

[1] "numero_de_cliente"                          
   [2] "foto_mes"                                   
   [3] "active_quarter"                             
   [4] "cliente_vip"                                
   [5] "internet"                                   
   [6] "cliente_edad"                               
   [7] "cliente_antiguedad"                         
   [8] "mrentabilidad"                              
   [9] "mrentabilidad_annual"                       
  [10] "mcomisiones"                                
  [11] "mactivos_margen"                            
  [12] "mpasivos_margen"                            
  [13] "cproductos"                                 
  [14] "tcuentas"                                   
  [15] "ccuenta_corriente"                          
  [16] "mcuenta_corriente_adicional"                
  [17] "mcuenta_corriente"                          
  [18] "ccaja_ahorro"                               
  [19] "mcaja_ahorro"                               
  [20] "mcaja_ahorro_adicional"                     
  [21] "mcaja_ahorro_dolares"                       
  [22] "cdescubierto_preacordado"                   
  [23] "mcuentas_saldo"                             
  [24] "ctarjeta_debito"                            
  [25] "ctarjeta_debito_transacciones"              
  [26] "mautoservicio"                              
  [27] "ctarjeta_visa"                              
  [28] "ctarjeta_visa_transacciones"                
  [29] "mtarjeta_visa_consumo"                      
  [30] "ctarjeta_master"                            
  [31] "ctarjeta_master_transacciones"              
  [32] "mtarjeta_master_consumo"                    
  [33] "cprestamos_personales"                      
  [34] "mprestamos_personales"                      
  [35] "cprestamos_prendarios"                      
  [36] "mprestamos_prendarios"                      
  [37] "cprestamos_hipotecarios"                    
  [38] "mprestamos_hipotecarios"                    
  [39] "cplazo_fijo"                                
  [40] "mplazo_fijo_dolares"                        
  [41] "mplazo_fijo_pesos"                          
  [42] "cinversion1"                                
  [43] "minversion1_pesos"                          
  [44] "minversion1_dolares"                        
  [45] "cinversion2"                                
  [46] "minversion2"                                
  [47] "cseguro_vida"                               
  [48] "cseguro_auto"                               
  [49] "cseguro_vivienda"                           
  [50] "cseguro_accidentes_personales"              
  [51] "ccaja_seguridad"                            
  [52] "cpayroll_trx"                               
  [53] "mpayroll"                                   
  [54] "mpayroll2"                                  
  [55] "cpayroll2_trx"                              
  [56] "ccuenta_debitos_automaticos"                
  [57] "mcuenta_debitos_automaticos"                
  [58] "ctarjeta_visa_debitos_automaticos"          
  [59] "mttarjeta_visa_debitos_automaticos"         
  [60] "ctarjeta_master_debitos_automaticos"        
  [61] "mttarjeta_master_debitos_automaticos"       
  [62] "cpagodeservicios"                           
  [63] "mpagodeservicios"                           
  [64] "cpagomiscuentas"                            
  [65] "mpagomiscuentas"                            
  [66] "ccajeros_propios_descuentos"                
  [67] "mcajeros_propios_descuentos"                
  [68] "ctarjeta_visa_descuentos"                   
  [69] "mtarjeta_visa_descuentos"                   
  [70] "ctarjeta_master_descuentos"                 
  [71] "mtarjeta_master_descuentos"                 
  [72] "ccomisiones_mantenimiento"                  
  [73] "mcomisiones_mantenimiento"                  
  [74] "ccomisiones_otras"                          
  [75] "mcomisiones_otras"                          
  [76] "cforex"             

### Reduccion dimensionalidad con canaritos asesinos

Intencionalmente este código no se brinda
<br> El objetivo de esto es reducir la dimensionalidad del dataset unicamente para que la Bayesian Optimization corra mas rápido
<br> Tambien se puede probar con Boruta

## Modelado

### Target Engineering

In [18]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)]

### Training Strategy

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202108  obviamente completo

* final_train =  [201901, 202106] sin undersampling de los CONTINUA

* training
   * testing = 202106
   * training = [201901, 202104]  donde se consideran el 5% de los CONTINUA

In [19]:
PARAM$trainingstrategy$testing <- c(202106)

PARAM$trainingstrategy$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104
)

PARAM$trainingstrategy$undersampling <- 0.05


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [20]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

#### Registros  cambio las proporciones de POS/NEG

In [21]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar := runif(nrow(dataset))]
dataset[, training := 0L]

dataset[  foto_mes %in%  PARAM$trainingstrategy$training &
  (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]


### Optimizacion de Hipeparámetros

Se optimizan los hiperparámetros maximizando la ganancia.

In [22]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

In [23]:
dtrain <- lgb.Dataset(
  data= data.matrix(dataset[training == 1L, campos_buenos, with = FALSE]),
  label= dataset[training == 1L, clase01],
  free_raw_data= TRUE
)

cat("filas", nrow(dtrain), "columnas", ncol(dtrain), "\n")
Sys.time()

filas 236042 columnas 1253 


[1] "2025-11-01 13:45:53 EDT"

In [24]:
# defino los datos de testing
dataset_test <- dataset[foto_mes %in% PARAM$trainingstrategy$testing]

# precalculo el campo de la ganancia
dataset_test[, gan := -20000.0 ]
dataset_test[ clase_ternaria=="BAJA+2", gan := 780000]

# precalculo la test_matrix
test_matrix <- data.matrix(dataset_test[, campos_buenos, with= FALSE])

cat("filas", nrow(dataset_test), "columnas", ncol(dataset_test), "\n")
Sys.time()

filas 164313 columnas 1258 


[1] "2025-11-01 13:45:56 EDT"

In [25]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames




In [ ]:
# Especificacion de la cantidad de iteraciones de la Bayesian Optimization
# 50 es razonable
PARAM$hipeparametertuning$BO_iteraciones <- 30 # un 50 seria mas razonable

In [26]:
# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "custom",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  extra_trees = FALSE,

  max_depth = -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split = 0.0, # min_gain_to_split >= 0.0
  min_sum_hessian_in_leaf = 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1 = 0.0, # lambda_l1 >= 0.0
  lambda_l2 = 0.0, # lambda_l2 >= 0.0

  bagging_fraction = 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction = 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction = 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance = FALSE, #
  scale_pos_weight = 1.0, # scale_pos_weight > 0.0

  drop_rate = 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop = 50, # <=0 means no limit
  skip_drop = 0.5, # 0.0 <= skip_drop <= 1.0

  max_bin= 31
)

In [27]:
# Notar que se recorren algunos hiperparametros en forma logaritmica
#   y que con forbidden se tiene en cuenta el juego que hay entre min_data_in_leaf y num_leaves

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeNumericParam("num_iterations", lower= 0.0, upper= 11.1, trafo= function(x) as.integer( round(2^x)) ),
  makeNumericParam("learning_rate", lower= -8.0, upper= -1.0, trafo= function(x) 2^x ),
  makeNumericParam("feature_fraction", lower= 0.05, upper= 1.0 ),
  makeNumericParam("min_data_in_leaf", lower= 0.0, upper= log2(nrow(dtrain)/2), trafo= function(x) as.integer(round(2^x)) ),
  makeNumericParam("num_leaves", lower= 1.0, upper= 10.0, trafo= function(x) as.integer(round(2^x)) ),
  forbidden= quote( (2^min_data_in_leaf)*(2^num_leaves) > nrow(dtrain) )
)

El parámetro **ksemillerio** indica se se hace semillerio DENTRO de la bayesiana
* 1 **no** se hace Ensemble Semillerio, apenas se corre un solo LightGBM
* mayor a 1, se hace un  k-Ensemble Semillerio

El parámetro  **repe** indica si dentro de la bayesiana se toman varias medidas y luego se promedian
<br> Esto se hace ya sea que se llama a un solo LightGBM o se haceun Ensemble Semillerio de LightGBMs
<br> Tener en cuenta que repe multiplica linealmente el tiempo de corrida de la Bayesian Optimization

In [28]:
PARAM$hipeparametertuning$ksemillerio <- 1L
PARAM$hipeparametertuning$repe <- 1L


In [29]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [30]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$BO$semillas <- sample(primos)[seq( PARAM$hipeparametertuning$ksemillerio * PARAM$hipeparametertuning$repe )]

cat( PARAM$BO$semillas)

974411

In [31]:
if(!require("rlist")) install.packages("rlist")
require("rlist")

Loading required package: rlist



In [32]:
# logueo al archivo BO_log.txt
loguear  <- function( reg, arch=NA, verbose=TRUE )
{
  t0 <- Sys.time()
  archivo <- arch
  if( is.na(arch) ) archivo <- paste0( folder, substitute( reg), ext )


  if( !file.exists( archivo ) )
  {
    # Escribo los titulos
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  # escribo el registro
  linea  <- paste0( format(t0, "%Y%m%d.%H%M%S"),  "\t",     # la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  # grabo al archivo

  if( verbose )  cat( linea )   # imprimo por pantalla
}

In [33]:
# esto esta en una funcion para que el garbage collector lo libere
# entrena, aplica el modelo a testing, y devuelve el vector de probabilidades

OneTrainPredict <- function(param_completo) {
    
  modelo <- lgb.train(
    data= dtrain,
    param= param_completo
  )
  gmodelo <<- modelo

  # aplico el modelo a los datos nuevos
  pred <- predict(
    modelo,
    test_matrix
  )

  return( pred )
}

In [34]:
# En el argumento x llegan los parmaetros de la bayesiana
#  devuelve la ganancia en datos de testing

# aqui se ira guardando la mejor iteracion de la bayesiana
gmejor <- list()
gmejor$iter <- 0
gmejor$gan <- -Inf

giter <- 0
if( file.exists("BO_log.txt") ){
  tb_BO <- fread("BO_log.txt")
  giter <- nrow(tb_BO) -1 
}

EstimarGanancia_lightgbm <- function(x) {

  giter <<- giter + 1
  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  vgan_mesetas <- c()  # las ganancias, tengo repe de ellas

  # loop de las repeticionies
  for( repe in seq( PARAM$hipeparametertuning$repe ) )
  {
     desde <- (PARAM$hipeparametertuning$repe-1)*PARAM$hipeparametertuning$ksemillerio + 1
     hasta <- desde + PARAM$hipeparametertuning$ksemillerio -1
     rsemillas <- PARAM$BO$semillas[ desde:hasta ]

     # vector inicial de probabilidades
     vpred_acum <- rep( 0.0, nrow(dataset_test) )

     # loop del semillerio
     for( sem in rsemillas ) # itero semillerio
     {
        param_completo$seed <- sem  # asigno se semilla
        vpred_acum <- vpred_acum + OneTrainPredict( param_completo )
        
        gc(full= TRUE, verbose= FALSE)
     }

     # Calculo de ganancia suavizada de la meseta
     tb_prediccion <- dataset_test[, list(gan)]
     tb_prediccion[, prob := vpred_acum ]

     setorder(tb_prediccion, -prob)
     tb_prediccion[, gan_acum := cumsum(gan)]

     # la meseta es un punto, mil para la izquierda, otros mil para la derecha
     tb_prediccion[, gan_meseta :=
       frollmean(
         x= gan_acum, n= 2001, align= "center",
         na.rm= TRUE, hasNA= TRUE
      )
     ]

     vgan_mesetas <- c(vgan_mesetas, tb_prediccion[, max(gan_meseta, na.rm = TRUE)] )
  }

  gan_mesetas_prom <- mean( vgan_mesetas ) 

  if( gan_mesetas_prom > gmejor$gan ){
    gmejor$gan <<- gan_mesetas_prom
    gmejor$iter <<- giter

    # hrabo importancia de variables
    fwrite( lgb.importance(gmodelo),
      file= paste0("impo_", giter, ".txt"),
      sep= "\t"
    )
  }

  # datos qeu voy a loguear
  xx <- copy(param_completo)
  xx$iter <- giter
  xx$metrica_mejor <- gmejor$gan
  xx$metrica_sd <- sd(vgan_mesetas)
  xx$metrica <- gan_mesetas_prom

  loguear( xx, "BO_log.txt")
  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")  # le reordeno a mlrMBO

  return( gan_mesetas_prom ) #tiempo_corrida) )
}

In [35]:
# Aqui comienza la configuracion de la Bayesian Optimization
#  es compleja la configuracion de una Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hipeparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hipeparametertuning$BO_iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)

Sys.time()

[1] "2025-11-01 13:45:58 EDT"

#### Corrida de la Bayesian Optimization

In [36]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Sys.time()

Computing y column(s) for design. Not provided.



20251101.134611	binary	auc	TRUE	TRUE	FALSE	-100	TRUE	974411	FALSE	-1	0	0.001	0	0	1	1	1	FALSE	1	0.1	50	0.5	31	9	0.00751513077819927	0.284858822397221	92	33	1	325766696.651674	NA	325766696.651674
20251101.134613	binary	auc	TRUE	TRUE	FALSE	-100	TRUE	974411	FALSE	-1	0	0.001	0	0	1	1	1	FALSE	1	0.1	50	0.5	31	2	0.0325416602472422	0.293619126074546	12	173	2	325766696.651674	NA	310205887.056472
20251101.134658	binary	auc	TRUE	TRUE	FALSE	-100	TRUE	974411	FALSE	-1	0	0.001	0	0	1	1	1	FALSE	1	0.1	50	0.5	31	452	0.177703738626204	0.67202623474562	1060	53	3	379287786.106947	NA	379287786.106947
20251101.134800	binary	auc	TRUE	TRUE	FALSE	-100	TRUE	974411	FALSE	-1	0	0.001	0	0	1	1	1	FALSE	1	0.1	50	0.5	31	542	0.0122104653103434	0.461279257690437	245	39	4	379287786.106947	NA	375330774.612694
20251101.134802	binary	auc	TRUE	TRUE	FALSE	-100	TRUE	974411	FALSE	-1	0	0.001	0	0	1	1	1	FALSE	1	0.1	50	0.5	31	11	0.0189202342506775	0.662187630768755	2	21	5	379287786.106947	NA	338521869.065467
20251101.134834	binary	auc	T

[mbo] 0: num_iterations=9; learning_rate=0.00752; feature_fraction=0.285; min_data_in_leaf=92; num_leaves=33 : y = 3.26e+08 : 13.8 secs : initdesign

[mbo] 0: num_iterations=2; learning_rate=0.0325; feature_fraction=0.294; min_data_in_leaf=12; num_leaves=173 : y = 3.1e+08 : 2.0 secs : initdesign

[mbo] 0: num_iterations=452; learning_rate=0.178; feature_fraction=0.672; min_data_in_leaf=1060; num_leaves=53 : y = 3.79e+08 : 44.8 secs : initdesign

[mbo] 0: num_iterations=542; learning_rate=0.0122; feature_fraction=0.461; min_data_in_leaf=245; num_leaves=39 : y = 3.75e+08 : 61.9 secs : initdesign

[mbo] 0: num_iterations=11; learning_rate=0.0189; feature_fraction=0.662; min_data_in_leaf=2; num_leaves=21 : y = 3.39e+08 : 1.8 secs : initdesign

[mbo] 0: num_iterations=783; learning_rate=0.0498; feature_fraction=0.614; min_data_in_leaf=44; num_leaves=7 : y = 3.66e+08 : 31.7 secs : initdesign

[mbo] 0: num_iterations=1; learning_rate=0.0629; feature_fraction=0.949; min_data_in_leaf=469; num_l

20251101.135416	binary	auc	TRUE	TRUE	FALSE	-100	TRUE	974411	FALSE	-1	0	0.001	0	0	1	1	1	FALSE	1	0.1	50	0.5	31	310	0.090676140919715	0.363505231129914	44	192	21	385210384.807596	NA	385210384.807596


[mbo] 1: num_iterations=310; learning_rate=0.0907; feature_fraction=0.364; min_data_in_leaf=44; num_leaves=192 : y = 3.85e+08 : 51.8 secs : infill_ei



...

20251101.171958	binary	auc	TRUE	TRUE	FALSE	-100	TRUE	974411	FALSE	-1	0	0.001	0	0	1	1	1	FALSE	1	0.1	50	0.5	31	2175	0.0299340128231829	0.0572088461869817	155	1024	49	403869625.187406	NA	403034622.688656


[mbo] 29: num_iterations=2175; learning_rate=0.0299; feature_fraction=0.0572; min_data_in_leaf=155; num_leaves=1024 : y = 4.03e+08 : 365.1 secs : infill_ei

Saved the current state after iteration 30 in the file bayesiana.RDATA.



20251101.173436	binary	auc	TRUE	TRUE	FALSE	-100	TRUE	974411	FALSE	-1	0	0.001	0	0	1	1	1	FALSE	1	0.1	50	0.5	31	2150	0.0375467240522661	0.954954534738659	1	1016	50	403869625.187406	NA	391850814.592704


[mbo] 30: num_iterations=2150; learning_rate=0.0375; feature_fraction=0.955; min_data_in_leaf=1; num_leaves=1016 : y = 3.92e+08 : 874.5 secs : infill_ei

Saved the final state in the file bayesiana.RDATA

Saved the final state in the file bayesiana.RDATA



[1] "2025-11-01 17:34:36 EDT"

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Obviamente los datos donde se aplica el modelo es el mes  {202108} , que no tiene clase
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202106]
* Se realiza undersampling al 10%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

* Por experimentos en meses anteriores, se decide cortar en los 11000 registros con mayor probabildiad de POS={"BAJA+1", "BAJA+2"}, , esta es una *enorme* decisión.

### Final Training Strategy

In [37]:
PARAM$train_final$future <- c(202108)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106
)

PARAM$train_final$undersampling <- 0.10

In [38]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

#### Registros cambio las proporciones de POS/NEG

In [39]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [40]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Adaptacion Hiperparametros Optimos

Solamente escalo min_data_in_leaf  por  nrow(dataset_train_final) / nrow(dtrain)

In [41]:
# leo el archivo donde quedaron los hiperparametros optimos
tb_BO <-  fread("BO_log.txt")
setorder( tb_BO, -metrica)  # ordeno por metrica descendente
tb_BO[1]

fecha,objective,metric,first_metric_only,boost_from_average,feature_pre_filter,verbosity,force_row_wise,seed,extra_trees,⋯,max_bin,num_iterations,learning_rate,feature_fraction,min_data_in_leaf,num_leaves,iter,metrica_mejor,metrica_sd,metrica
<dbl>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<int>,<lgl>,<int>,<lgl>,⋯,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<lgl>,<dbl>
20251101,binary,auc,TRUE,TRUE,FALSE,-100,TRUE,974411,FALSE,⋯,31,2149,0.003916348,0.4570216,231,1021,40,403869625,NA,403869625


In [42]:
# en la tabla ademas de los parametros del LightGBM, hay campos de salida
param_lgbm <- union( names(PARAM$lgbm$param_fijos),  names(PARAM$hipeparametertuning$hs$pars) )

PARAM$train_final$param_mejores <- as.list( tb_BO[1, param_lgbm, with=FALSE])

PARAM$train_final$param_mejores$min_data_in_leaf <- as.integer( round(PARAM$train_final$param_mejores$min_data_in_leaf * nrow(dataset_train_final[training == 1L]) / nrow(dtrain)))

cat( tb_BO[1, min_data_in_leaf] , PARAM$train_final$param_mejores$min_data_in_leaf, "\n")
PARAM$train_final$param_mejores

231 464 


$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] TRUE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$verbosity
[1] -100

$force_row_wise
[1] TRUE

$seed
[1] 974411

$extra_trees
[1] FALSE

$max_depth
[1] -1

$min_gain_to_split
[1] 0

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0

$lambda_l2
[1] 0

$bagging_fraction
[1] 1

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$max_bin
[1] 31

$num_iterations
[1] 2149

$learning_rate
[1] 0.003916348

$feature_fraction
[1] 0.4570216

$min_data_in_leaf
[1] 464

$num_leaves
[1] 1021

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

In [43]:
# Semillerio Final
PARAM$train_final$ksemillerio  <- 30


In [44]:
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$ksemillerio )]
PARAM$train_final$semillas

[1] 974411 507673 696271 281233 677309 180647 251519 235241 176213 162601
[11] 138637 519551 807931 471301 152063 823663 874337 466201 206191 787939
[21] 361327 320141 459847 246833 208799 381371 868327 745187 297371 244147

In [45]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 473690 columnas 1253 


[1] "2025-11-01 17:34:51 EDT"

In [46]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$param_mejores)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
     gc()
  }
}

Sys.time()

[1] "2025-11-02 00:42:12 EDT"

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [47]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]
mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

vpred_acum <- rep(0.0, nrow(dfuture))
qacumulados <- 0

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( file.exists( arch_modelo ) )
  {
    modelo_final <- lgb.load(arch_modelo) # leo del disco
    #hago el predict() y acumulo
    vpred_acum <- vpred_acum + predict(modelo_final, mfuture)
    qacumulados <- qacumulados + 1
  }
}

vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
Sys.time()

[1] "2025-11-02 00:49:19 EDT"

In [48]:
# tabla de prediccion, puede ser util para futuros ensembles
#  ya que le modelo ganador va a ser un ensemble de LightGBMs

tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := vpred_acum ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [49]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

envios <- 11000
tb_prediccion[, Predicted := 0L] # seteo inicial a 0
tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
  file= archivo_kaggle,
  sep= ","
)

In [50]:
Sys.time()

[1] "2025-11-02 00:49:19 EDT"